In [15]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# load model
with open('../rf_test_model.pkl', 'rb') as model:
    rf = pickle.load(model)

# load data
blocks = pd.read_csv("../blocks.csv")
transactions = pd.read_csv("../transactions.csv")

df = pd.read_csv("../blocks.csv")
df2 = pd.read_csv("../transactions.csv")

/Users/tracy/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/Users/tracy/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [14]:
df2_agg = df2[['block_timestamp', 'receipt_effective_gas_price', 'max_priority_fee_per_gas']].groupby('block_timestamp').agg(['mean', 'count'])
df2_agg.columns = df2_agg.columns.map('_'.join).str.strip('_')
df_merge = df.merge(right=df2_agg, how='inner', on='block_timestamp')
df_merge = pd.DataFrame(df_merge).sort_values(by='block_timestamp', ascending=True)
df_merge = df_merge[-200:] # need 200 to do 100-ago division on last of 100 most recent

cols = ['base_fee_per_gas', 'receipt_effective_gas_price_count', 'receipt_effective_gas_price_mean', 'max_priority_fee_per_gas_mean']
for col in cols:
    # Last 5 blocks
    df_merge[col+'_pct_chg_last_5'] = df_merge[col]/df_merge[col].shift(5)-1
    # 100 blocks ago to 5 blocks ago percentage changes
    df_merge[col+'_pct_chg_last_100_to_5'] = df_merge[col].shift(5)/df_merge[col].shift(100)-1

df_merge = df_merge[-100:] # only want those 100 most recent which have complete percents

# datetime dummy variables
df_merge['local_date'] = pd.to_datetime(df_merge['block_timestamp']).dt.tz_localize('utc').dt.tz_convert('US/Eastern')
df_merge['date'] = df_merge['local_date'].dt.date
df_merge['hour'] = df_merge['local_date'].dt.hour
df_merge['minute'] = df_merge['local_date'].dt.minute
df_merge['weekday'] = df_merge['local_date'].dt.weekday

df_merge['hour_dummy'] = 0
start_hour = 2
end_hour = 9
mask = (df_merge['hour'] < start_hour) | (df_merge['hour'] > end_hour)
df_merge.loc[mask, 'hour_dummy'] = 1

df_merge['minute_dummy'] = 0
start_minute = 1
end_minute = 6
mask = (df_merge['minute'] >= start_minute) & (df_merge['minute'] <= end_minute)
df_merge.loc[mask, 'minute_dummy'] = 1

df_merge['weekday_dummy'] = 0
start_weekday = 1
end_weekday = 4
mask = (df_merge['weekday'] >= start_weekday) & (df_merge['weekday'] <= end_weekday)
df_merge.loc[mask, 'weekday_dummy'] = 1

# get only columns needed
features = ['base_fee_per_gas_pct_chg_last_100_to_5', 
            'base_fee_per_gas_pct_chg_last_5', 
            'receipt_effective_gas_price_count_pct_chg_last_100_to_5', 
            'receipt_effective_gas_price_count_pct_chg_last_5', 
            'receipt_effective_gas_price_mean_pct_chg_last_100_to_5', 
            'receipt_effective_gas_price_mean_pct_chg_last_5', 
            'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5', 
            'max_priority_fee_per_gas_mean_pct_chg_last_5', 
            'minute_dummy', 'hour_dummy', 'weekday_dummy'
            ]
df_predict = df_merge[features]

# swap out infs (generated during % change step if denominator is 0)
columns = df_predict.columns
values = [0.431637, 0.309303, 8.809524, 8.9, 17.677177, 16.710033, 15.495496, 10.24462]
for i in columns[:8]:
    df_predict[i].replace(np.inf, values[columns.get_loc(i)], inplace=True)

####################
# make predictions #
####################

predicted = rf.predict(df_predict)

/Users/tracy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/tracy/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [10]:
#df2_agg

In [11]:
#transactions.tail(100)['receipt_effective_gas_price']

In [12]:
#transactions['predicted'] = 

In [13]:
blocks

,block_timestamp,base_fee_per_gas
0,2021-11-26 21:32:09,116925004757
1,2021-11-26 21:31:58,121935382610
2,2021-11-26 21:31:38,108389042873
3,2021-11-26 21:31:36,123043231276
4,2021-11-26 21:31:28,118132043184
...,...,...
245,2021-11-26 20:32:48,101632253377
246,2021-11-26 20:32:43,111328001027
247,2021-11-26 20:32:32,109482183896
248,2021-11-26 20:32:15,97330091311
